# 파이프라인 사용

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
import joblib

df = pd.read_csv('diabetes_data.csv')
df.info()
df
# 1) 데이터 준비
#    - df: 컬럼 ['Age','Gender','Blood_Pressure','Cholesterol','Diabetes'] 가정
#    - 타깃 y는 Yes/No -> 1/0으로 매핑
#    - X는 원본 문자열 Gender 그대로 두고 파이프라인에서 인코딩
# ------------------------------------------------------------
y = df['Diabetes'].map({'Yes': 1, 'No': 0})
X = df.drop(columns=['Diabetes'])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=11, stratify=y
)

# 2) 컬럼별 전처리 정의
#    - 수치형: StandardScaler
#    - 범주형(Gender): OneHotEncoder(drop='if_binary') -> 이진이면 1개 열로 깔끔하게
# ------------------------------------------------------------
num_features = ['Age', 'Blood_Pressure', 'Cholesterol']
cat_features = ['Gender']

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_features),
        ("cat", OneHotEncoder(drop='if_binary', handle_unknown='ignore'), cat_features),
    ],
    remainder='drop'  # 명시 컬럼만 사용
)

# 3) 전처리 + 모델 파이프라인
# ------------------------------------------------------------
base_pipe = Pipeline([
    ("pre", preprocessor),
    ("svm", SVC(kernel='linear', C=10, random_state=11))  # probability=False여도 OK (결정함수 사용)
])

# 4) 확률 보정(Platt scaling) 래핑
#    - CalibratedClassifierCV가 내부 CV로 base_pipe를 복제/학습하여 보정 파라미터를 학습
#    - base_pipe를 미리 fit()하지 않고, 보정기에서 fit()을 수행해야 함
# ------------------------------------------------------------
calibrated_clf = CalibratedClassifierCV(
    estimator=base_pipe,
    method='sigmoid',  # Platt scaling
    cv=5               # 보정 파라미터 추정을 위한 내부 CV
)

# 학습 (전처리+모델 학습+보정이 한 번에 이루어짐)
calibrated_clf.fit(X_train, y_train)

# 5) 예측 및 평가
# ------------------------------------------------------------
y_pred = calibrated_clf.predict(X_test)

print("Confusion Matrix")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report")
print(classification_report(y_test, y_pred, digits=4))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Age             1000 non-null   int64 
 1   Gender          1000 non-null   object
 2   Blood_Pressure  1000 non-null   int64 
 3   Cholesterol     1000 non-null   int64 
 4   Diabetes        1000 non-null   object
dtypes: int64(3), object(2)
memory usage: 39.2+ KB
Confusion Matrix
[[ 55   6]
 [  0 239]]

Classification Report
              precision    recall  f1-score   support

           0     1.0000    0.9016    0.9483        61
           1     0.9755    1.0000    0.9876       239

    accuracy                         0.9800       300
   macro avg     0.9878    0.9508    0.9679       300
weighted avg     0.9805    0.9800    0.9796       300

